# Hyperparameter Tuning

This notebook determines the best set of hyperparameters to train the portfolio optimization model.

#### Import necessary modules

In [31]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import boto3
from IPython.display import Image
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
from sagemaker.session import Session
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
import shutil


#### Set local parameters

In [32]:
image_type = 'cpu'
instance_type = 'ml.m5.large'
start_day = 2670
n_instances = 1

#### Compile environment

In [33]:
role = get_execution_role()
account = boto3.client('sts').get_caller_identity()['Account']
region = boto3.Session().region_name
image_name = '{}.dkr.ecr.{}.amazonaws.com/portfolio-optimization-{}:latest'.format(account, region, image_type)
print(image_name)

031118886020.dkr.ecr.us-east-1.amazonaws.com/portfolio-optimization-cpu:latest


#### Set the objective
Next we'll specify the objective metric that we'd like to tune and its definition, which includes the regular expression (Regex) needed to extract that metric from the CloudWatch logs of the training job. In this particular case, our script emits the objective metric directly, which we will 'Maximize'.  The actual objective metric $O$ is defined below.

$
\qquad O = 100*(\frac{P}{M} - 1) - max(0, t-60) \\
\qquad \qquad \text{where:} \\
\qquad \qquad \qquad P = \text{The final Portfolio value} \\
\qquad \qquad \qquad M = \text{The final Market value} \\
\qquad \qquad \qquad t = \text{The total training time in minutes} \\
$

In [34]:
objective_metric_name = 'Trading Performance'
objective_type = 'Maximize'
metric_definitions = [{'Name': objective_metric_name,
                       'Regex': '(\S+) training objective.'}]

#### Define the hyperparameters to optimize
The following hyperparameters are read from the command line in [train.py](container/src/train.py).

| Name            | Type  | Default  | Description                        |
|-----------------|-------|----------|------------------------------------|
| max_epochs      | int   |    2000  | max epochs per new trading day     |
| days_per_epoch  | int   |      40  | days in each epoch                 |
| start_day       | int   |     504  | day to begin training              |
| window_length   | int   |       1  | CNN window length                  |
| memory_strength | float |     2.0  | memory exponential gain            |
| target          | float |    1.02  | target portfolio/market ratio      |
| fc1             | int   |     128  | size of 1st hidden layer           |
| fc2             | int   |      64  | size of 2bd hidden layer           |
| lr_actor        | float | 0.000371 | initial learning rate for actor    |
| lr_critic       | float |   0.0011 | initial learning rate for critic   |
| batch_size      | int   |      256 | mini batch size                    |
| buffer_size     | int   |   100000 | replay buffer size                 |
| gamma           | float |     0.91 | discount factor                    |
| tau             | float |   0.0072 | soft update of target parameters   |
| sigma           | float |    0.013 | OU Noise standard deviation        |

Any of these could be tuned but we will down select to limit the search.    

The hyperparameter tunner allow the hyperparameters to be defined as one of the following types. 
- `CategoricalParameter(list)` Categorical parameters need to take one value from a discrete set. 
- `ContinuousParameter(min, max)` Continuous parameters can take any real number value between the minimum and maximum value.
- `IntegerParameter(min, max)` Integer parameters can take any integer value between the minimum and maximum value.

_Note, if possible, it's almost always best to specify a value as the least restrictive type. For example, tuning learning rate as a continuous value between 0.01 and 0.2 is likely to yield a better result than tuning as a categorical parameter with values 0.01, 0.1, 0.15, or 0.2. Sometimes a parameter is categorical to limit the search space._

_Also parameters maybe group a sequence of optimization may be executed._

In [35]:
hyperparameter_ranges = {
    'days_per_epoch': IntegerParameter(40, 252),
    'fc1': IntegerParameter(256, 512),
    'fc2': IntegerParameter(128, 256),
    'lr_actor': ContinuousParameter(0.0001, 0.001),
    'lr_critic': ContinuousParameter(0.0005, 0.005)}

#### Create the base estimator

In [36]:
estimator = Estimator(role=role,
                  train_instance_count=n_instances,
                  train_instance_type=instance_type,
                  image_name=image_name,
                  hyperparameters={'start_day': start_day})

#### Create the hyperparameter tuner object

In [37]:
tuner = HyperparameterTuner(estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=100,
                            max_parallel_jobs=len(hyperparameter_ranges) + 1,
                            objective_type=objective_type)

#### Perform the hyperparameter tuning
After the hyperprameter tuning job is created, you should be able to describe the tuning job to see its progress in the next step, and you can go to SageMaker console -> `Training` -> `Hyperparameter tuning jobs` to see the progresss.

In [ ]:
tuner.fit()
tuner.wait()

......................................................

In [ ]:
best_parameters = tuner.best_estimator().hyperparameters()


In [ ]:
best_name = tuner.best_training_job()
print('\nBest model = {}.'.format(best_name))
for name in hyperparameter_ranges.keys():
    print('\t{} = {}'.format(name, best_parameters[name]))

#### Compare optimal hyperparameters to manually optimized values.
| Name        | Type  | Default | Description                        |
|-------------|-------|---------|------------------------------------|
| fc1         | int   |     128 | size of 1st hidden layer           |
| fc2         | int   |      64 | size of 2bd hidden layer           |
| lr_actor    | float |   0.001 | initial learning rate for actor    |
| lr_critic   | float |   0.001 | initial learning rate for critic   |
| batch_size  | int   |     256 | mini batch size                    |
| buffer_size | int   |  100000 | replay buffer size                 |
| gamma       | float |     0.9 | discount factor                    |
| tau         | float |   0.001 | soft update of target parameters   |
| sigma       | float |    0.01 | OU Noise standard deviation        |

# Review the Results

#### Copy and unpack the optimum result archive

In [ ]:
sagemaker_session = Session()
bucket = sagemaker_session.default_bucket()
s3 = boto3.resource('s3')
s3.Bucket(bucket).download_file('{}/output/output.tar.gz'.format(best_name), 'output.tar.gz')
shutil.unpack_archive('output.tar.gz')

#### View the optimial results

In [ ]:
Image(filename='history.png') 

# Reference
- [SageMaker Tuning Example](https://github.com/awslabs/amazon-sagemaker-examples/tree/master/hyperparameter_tuning/pytorch_mnist)
- [How Hyperparameter Tuning Works](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-how-it-works.html)
- [Tuner API](https://sagemaker.readthedocs.io/en/stable/api/training/tuner.html)
- [Estimater API](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase)